<a href="https://colab.research.google.com/github/brokentwigs/Colab-Analysis/blob/main/DecisionTrees_vs_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time

# visualizatoin
import matplotlib.pyplot as plt
import seaborn as sns

# data wrangling
import pandas as pd
import numpy as np

# data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.compose import make_column_transformer
import category_encoders as ce

#feature selection algorithms
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import roc_auc_score, roc_curve



#machine learning algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression



from xgboost import XGBClassifier


from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold, KFold, RandomizedSearchCV,GridSearchCV
import xgboost

#optimization
from numpy import sqrt
from numpy import argmax







import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: ignored

In [ ]:
!pip install category_encoders
#!pip install catboost
#!pip install scikit-optimize i

#run these pip installations first


In [ ]:
# description
description = pd.read_csv('/content/WiDS_Datathon_2020_Dictionary.csv')
description_dict = description.set_index('Variable Name').to_dict(orient='index')
# data
df = pd.read_csv('/content/training_v2.csv')

#print_full(description)
df.head()

In [ ]:
# Returns category description

def category(description, string):

    return description[description.Category==string]

# Returns highly correlated features
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr


In [ ]:
# Parameter definitions
test_size = 0.2 # proportion for train versus test+val split
val_size = 0.5 # proportion for test versus val split
random_state = 42  # random state is used to set a seed for randomness, which is only relevant for reproducibility purposes
max_missing = 0.8  # maximum percentage of missing values for a column to be dropped

In [ ]:
X = df.copy().drop(['hospital_death'], axis=1)
y = df['hospital_death'].copy()
y_apache = df['apache_4a_hospital_death_prob'].copy() #save APACHE scores for later evaluation on train / test / validation data


""" SPLIT DATA SET """
# split the dataset into train and test+validation set
(
    X_train,
    X_test,
    y_train,
    y_test,
    y_apache_train,
    y_apache_test,
    ) = train_test_split(X, y, y_apache,
                         test_size=test_size, # used for testing and validation
                         random_state=random_state # for reproducibility
                        )
# split the test set into test + validation set
(
    X_val,
    X_test,
    y_val,
    y_test,
    y_apache_val,
    y_apache_test,
    ) = train_test_split(X_test, y_test, y_apache_test,
                         test_size=val_size, # used for testing and validation
                         random_state=random_state # for reproducibility
                        )

In [ ]:
apache_cov = category(description,"APACHE covariate")
vitals = category(description,"vitals")
labs = category(description,"labs")

corr_features_apache_cov = correlation(X_train[apache_cov["Variable Name"]], 0.85)
corr_features_vitals = correlation(X_train[vitals["Variable Name"]], 0.85)
corr_features_labs = correlation(X_train[labs["Variable Name"]], 0.85)

In [ ]:
# DROP THESE COLUMNS
to_drop = set()
to_drop.update(['apache_2_bodysystem', 'apache_4a_icu_death_prob', 'encounter_id', 'patient_id', 'weight', 'height',
                'icu_stay_type', 'readmission_status', 'hospital_id', 'icu_id',
                'apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob'])
to_drop.update(corr_features_apache_cov)
to_drop.update(corr_features_vitals)
to_drop.update(corr_features_labs)

In [ ]:
X_train = X_train.drop(to_drop, axis=1)
X_val = X_val.drop(to_drop, axis=1)
X_test = X_test.drop(to_drop, axis=1)



In [ ]:
"""MISSING VALUES"""
# drop columns with many missing values
missing = X_train.isna().sum() > max_missing * len(X_train)
missing = missing[missing].index
X_train = X_train.drop(missing, axis=1)
X_val = X_val.drop(missing, axis=1)
X_test = X_test.drop(missing, axis=1)

In [ ]:
def customprocessor(X, y):
    num = [col for col in X.columns if X[col].dtypes != 'O' and ~X[col].isin([0.0,1.0,np.nan]).all()]
    cat = [col for col in X.columns if X[col].dtypes == 'O' or X[col].isin([0.0,1.0,np.nan]).all()]

    # Imputing numerical values
    linearreg = LinearRegression()
    imp = IterativeImputer(estimator=linearreg, missing_values=np.nan, max_iter=5, imputation_order='roman', verbose=2, random_state=42)
    X[num] = imp.fit_transform(X[num])

    # Imputing categorical values
    impcat = SimpleImputer(strategy="most_frequent")
    X[cat] = impcat.fit_transform(X[cat])

    # Turn 'gender' column into a binary column
    gender = {'M': 1,'F': 0}
    X.gender = [gender[item] for item in X.gender]

    # Target encoding with additive smoothing
    def calc_smooth_mean(X, y, by, on, m):
        df = pd.concat([X, y], axis=1)

        # Compute the global mean
        mean = df[on].mean()

        # Compute the number of values and the mean of each group
        agg = df.groupby(by)[on].agg(['count', 'mean'])
        counts = agg['count']
        means = agg['mean']

        # Compute the "smoothed" means
        smooth = (counts * means + m * mean) / (counts + m)

        # Replace each value by the according smoothed mean
        return X[by].map(smooth)

    X['ethnicity'] = calc_smooth_mean(X, y, by='ethnicity', on='hospital_death', m=10)
    X['hospital_admit_source'] = calc_smooth_mean(X, y, by='hospital_admit_source', on='hospital_death', m=10)
    X['icu_admit_source'] = calc_smooth_mean(X, y, by='icu_admit_source', on='hospital_death', m=10)
    X['icu_type'] = calc_smooth_mean(X, y, by='icu_type', on='hospital_death', m=10)
    X['apache_3j_bodysystem'] = calc_smooth_mean(X, y, by='apache_3j_bodysystem', on='hospital_death', m=10)

    return X

In [ ]:
# Applying pre-processing steps
X_train = customprocessor(X_train, y_train)
X_val = customprocessor(X_val, y_val)
X_test = customprocessor(X_test, y_test)

In [ ]:
#Candidate algorithms. Describe the machine learning algorithms you will try (e.g., logistic regression, decision tree, random forest, XGBoost). For each algorithm, describe possible
#advantages and disadvantages, given the problem requirements.
#Model Selection. Use the algorithm(s) to train machine learning models (one for each al- gorithm).
#Describe your model selection pipeline, including the model selection procedure
#(hyperparameter tuning procedure, cross-validation, etc.) and the evaluation metric(s) that you use during model selection.


In [ ]:
def random_search(model, X, Y, n_splits=10, n_repeats=3):

    """
  Random search for hyperparameter tuning:
  model - Random Forest classifier, DecisionTrees
  X - Data matrix X, typically training data
  Y - Target vector Y, typically training labels/binary
  n_splits - splits to be made in repeated stratified kfold cv
  n_repeats - number of times to repeat stratified kfold cv
  Note --> The Search spaces are preset/defined randomly
      """
    if type(model) == type(RandomForestClassifier()):
        params = dict(
        n_estimators= [100,150],
        criterion = ['gini', 'entropy'],
        min_samples_split = [2, 3, 4],
        min_samples_leaf = [1, 2, 3, 4, 5],
        max_features = ['auto', 'sqrt'],
        max_depth = [5, 6, 7]
        )

    elif type(model) == type(DecisionTreeClassifier()):
        params = dict(
        criterion = ['gini', 'entropy'],
        min_samples_split = [2, 3, 4],
        min_samples_leaf = [1, 2, 3, 4, 5],
        max_depth = [5, 6, 7])



    elif type(model) == type(xgboost.XGBClassifier()):
        params = dict(gamma = [0.1, 0.5, 2, 6, 25, 100],
                      learning_rate = [0.01, 0.03, 0.06, 0.1, 0.2, 0.3],
                      max_depth = [5, 10, 15, 20],
                      n_estimators = [50, 80, 115, 150],
                      reg_alpha = [0.1, 0.5, 2, 6, 25, 100],
                      reg_lambda = [0.1, 0.5, 2, 6, 25, 100])



      # define evaluation
    cv = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)


    #random search

    search = RandomizedSearchCV(model, params,  n_iter=100, scoring='accuracy', n_jobs=-1, cv=cv)


    # execute search
    result = search.fit(X, Y)
    print('SEARCH COMPLETE')



    # return the best result
    return result.best_score_ , result.best_params_






In [ ]:
 def decision_classifier(X_train,y_train):
    params={
    'min_samples_split':3,
    'min_samples_leaf':3,
    'max_depth':6,
    'criterion':'gini'}

    w = {0:9, 1:91}


    dt = DecisionTreeClassifier(**params, class_weight = w)
    dt.fit(X_train, y_train)
    dt_preds = dt.predict(X_test)

    print("Accuracy TEST SET:",metrics.accuracy_score(y_test, dt_preds))


    print("The precision, recall and f1 score:", precision_recall_fscore_support(y_test, dt_preds, average='binary'))
    print('The balanced accuracy:',balanced_accuracy_score(y_test, dt_preds))

decision_classifier(X_train,y_train)

In [ ]:
 #sum of weight for the xgboost classifier, he does not have the class weight
y_train_0 = []
y_train_1 = []
for i in y_train:
    if i == 0:
        y_train_0.append(i)
    elif i == 1:
        y_train_1.append(i)

weight = len(y_train_0) / len(y_train_1)
weight

def xg_boost(X_train,y_train):
    params = params={
    'reg_lambda':25,
    'reg_alpha':2,
    'n_estimators':115,
    'max_depth': 20,
    'learning_rate':0.1,
    'gamma':6}


    xg = XGBClassifier(**params, scale_pos_weight=weight) # Init model
    xg.fit(X_train, y_train) # Fit model
    xg_preds = xg.predict(X_test.astype(int)) # Get model inference

    print("Accuracy TEST SET:",metrics.accuracy_score(y_test, xg_preds))


    print("The precision, recall and f1 score:", precision_recall_fscore_support(y_test, xg_preds, average='binary'))
    print('The balanced accuracy:',balanced_accuracy_score(y_test, xg_preds))

xg_boost(X_train.astype(int),y_train)


In [ ]:
def random_forest(X_train,y_train):
  params = {'n_estimators': 150,
  'min_samples_split': 2,
  'min_samples_leaf': 4,
  'max_features': 'auto',
  'max_depth': 7,
  'criterion': 'gini'}

  w = {0:9, 1:91}


  rf = RandomForestClassifier(**params, class_weight = w)
  rf.fit(X_train, y_train)
  rf_preds = rf.predict(X_test)

  print("Accuracy TEST SET:",metrics.accuracy_score(y_test, rf_preds))


  print("The precision, recall and f1 score:", precision_recall_fscore_support(y_test, rf_preds, average='binary'))
  print('The balanced accuracy:',balanced_accuracy_score(y_test, rf_preds))

random_forest(X_train,y_train)

ROC CURVE FOR ALL THE MACHINE LEARNING MODELS


In [ ]:
#ROC Curve for every trained model, and adjust the decision treshold

#ROC curve for all the trained models
params={
    'min_samples_split':3,
    'min_samples_leaf':3,
    'max_depth':6,
    'criterion':'gini'}

w = {0:9, 1:91}


dt = DecisionTreeClassifier(**params, class_weight = w)
dt.fit(X_train, y_train)
probs_dt = dt.predict_proba(X_test)[:, 1]


In [ ]:
params = {'n_estimators': 150,
  'min_samples_split': 2,
  'min_samples_leaf': 4,
  'max_features': 'auto',
  'max_depth': 7,
  'criterion': 'gini'}

w = {0:9, 1:91}


rf = RandomForestClassifier(**params, class_weight = w)
rf.fit(X_train, y_train)
probs_rf = rf.predict_proba(X_test)[:,1]


In [ ]:
params={
    'reg_lambda':25,
    'reg_alpha':2,
    'n_estimators':115,
    'max_depth': 20,
    'learning_rate':0.1,
    'gamma':6}


#scale_pos_ = 10.594500632111252
xg = XGBClassifier(**params, scale_pos_weight=weight) # Init model
xg.fit(X_train.astype(int), y_train) # Fit model
probs_xg = xg.predict_proba(X_test.astype(int))[:,1] # Get model inference

In [ ]:
w = {0:9, 1:91}
lr = LogisticRegression(random_state=random_state, class_weight=w)
lr.fit(X_train, y_train)
probs_lr = lr.predict_proba(X_test)[:, 1]

In [ ]:
auc_lr = roc_auc_score(y_test, probs_lr)
fpr_lr, tpr_lr, thresholds_lr = roc_curve(y_test, probs_lr)

auc_dt = roc_auc_score(y_test, probs_dt)
fpr_dt, tpr_dt, thresholds_dt = roc_curve(y_test, probs_dt)

auc_rf = roc_auc_score(y_test, probs_rf)
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, probs_rf)

auc_xg = roc_auc_score(y_test, probs_xg)
fpr_xg, tpr_xg, thresholds_xg = roc_curve(y_test, probs_xg)

plt.figure(figsize=(12, 7))
plt.plot(fpr_lr, tpr_lr, label=f'AUC (Logistic Regression) = {auc_lr:.2f}')
plt.plot(fpr_dt, tpr_dt, label=f'AUC (Decision Tree) = {auc_dt:.2f}')
plt.plot(fpr_rf, tpr_rf, label=f'AUC (Random Forests) = {auc_rf:.2f}')
plt.plot(fpr_xg, tpr_xg, label=f'AUC (XGBoost) = {auc_xg:.2f}')
plt.plot([0, 1], [0, 1], color='blue', linestyle='--', label='Baseline')
plt.title('ROC Curve', size=20)
plt.xlabel('False Positive Rate', size=14)
plt.ylabel('True Positive Rate', size=14)
plt.legend();

Change the decision treshold based on our problem statement

In [ ]:
#adjust the tresholds for all the trained models, based on the ROC curves
# calculate roc curves

optimial_dt = np.argmax(tpr_dt - fpr_dt)

best_thresh_dt = thresholds_dt[optimial_dt]

print('Best Threshold=%f for decision tree' % (best_thresh_dt))

# get the best threshold for logistic regression
J_lr = tpr_lr - fpr_lr
ix_lr = argmax(J_lr)
best_thresh_lr = thresholds_lr[ix_lr]
print('Best Threshold=%f for logistic regression' % (best_thresh_lr))


# get the best threshold for xgboost
J_xg = tpr_xg - fpr_xg
ix_xg = argmax(J_xg)
best_thresh_xg = thresholds_xg[ix_xg]
print('Best Threshold=%f for xgboost' % (best_thresh_xg))



# get the best threshold for random forest
J_rf = tpr_rf - fpr_rf
ix_rf = argmax(J_rf)
best_thresh_rf = thresholds_rf[ix_rf]
print('Best Threshold=%f for random forest' % (best_thresh_rf))

In [ ]:
probs_xg = xg.predict_proba(X_val.astype(int))
probs_xg_1 = probs_xg[:, 1]
probs_xg_new = (probs_xg_1 >= 0.172782).astype(int)

print("Accuracy TEST SET:",metrics.accuracy_score(y_val, probs_xg_new))
print("The precision, recall and f1 score:", precision_recall_fscore_support(y_val, probs_xg_new, average='binary'))
print('The balanced accuracy:',balanced_accuracy_score(y_val, probs_xg_new))